In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from particle import literals as lp

import DarkNews as dn

from DarkNews import const

from DarkNews.GenLauncher import GenLauncher

from DarkNews import Cfourvec as Cfv

In [4]:
if 0:
    print('a')

In [29]:
# targets
proton = dn.detector.NuclearTarget("H1")
C12 = dn.detector.NuclearTarget("C12")
Ar40 = dn.detector.NuclearTarget("Ar40")

In [30]:
bsm_model = dn.model.create_3portal_model(mzprime=0.1, m4 = 0.01, epsilon=0.0, theta=1e-4, s_mu4=1e-3)

Model:
	1 majorana heavy neutrino(s).
	higgs mixed h'
	1 non-zero scalar-neutrino coupling(s).


In [31]:
bsm_model.cSproton

1.1470964962289811e-07

In [32]:
dn.model.FermionDileptonDecay(dn.pdg.neutrino5, dn.pdg.neutrino4, lp.e_minus, lp.e_plus, bsm_model)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [34]:
dn.pdg.neutrino4.mass

0.01